In [ ]:
!pip install huggingface_hub[hf_xet]

import os, json, glob
from huggingface_hub import hf_hub_download
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output
import shutil

# HuggingFace Token
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN", None)

# *.mlst を検索
mlst_files = glob.glob("**/*.mlst", recursive=True)
mlst_files = [os.path.relpath(p, ".") for p in mlst_files]

# 出力
out = widgets.Output()

# ドロップダウン
mlst_dd = widgets.Dropdown(
    options=mlst_files if mlst_files else ["--- 見つからない ---"],
    description="mlst:",
    layout=widgets.Layout(width="50%"),
    style={'description_width': 'initial'}
)

# ボタン
btn_download = widgets.Button(description="ダウンロード", button_style="info", icon="download")
btn_delete   = widgets.Button(description="削除", button_style="warning", icon="trash-o")

# ダウンロード処理
def on_download(_):
    with out:
        out.clear_output()
        json_file = mlst_dd.value
        print(f"▶ {json_file} を読み込み中...")

        try:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)
            repos = data.get("repositories", [])
        except Exception as e:
            print(f"❌ mlst読込失敗: {e}")
            return

        for repo in repos:
            repo_id = repo["repo_id"]
            files = repo.get("files", [])
            save_dir = repo.get("save_path", "./downloads")
            os.makedirs(save_dir, exist_ok=True)

            for filename in files:
                filename_only = os.path.basename(filename)
                target_path = os.path.join(save_dir, filename_only)
                if os.path.exists(target_path):
                    print(f"⏩ {target_path} は既に存在")
                    continue
                try:
                    print(f"⬇ {filename} を {repo_id} からDL中...")
                    downloaded = hf_hub_download(
                        repo_id=repo_id,
                        filename=filename,
                        token=HF_TOKEN,
                        force_download=True,
                    )
                    real_path = os.path.realpath(downloaded)  # リンクを解決して実体のパスを取得
                    shutil.move(real_path, f"{HOME_DIR}/{target_path}")    # 実体を移動
                    print(f"✅ {filename} ダウンロード完了")
                except Exception as e:
                    print(f"❌ {filename} 失敗: {e}")
        print("✅リストのモデルを全てダウンロードしました。")
        

# 削除処理
def on_delete(_):
    with out:
        out.clear_output()
        json_file = mlst_dd.value
        print(f"▶ {json_file} に記載されたファイルを削除します")

        try:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)
            repos = data.get("repositories", [])
        except Exception as e:
            print(f"❌ JSON読込失敗: {e}")
            return

        for repo in repos:
            save_dir = repo.get("save_path", "./downloads")
            files = repo.get("files", [])
            for filename in files:
                filename_only = os.path.basename(filename)
                target_path = os.path.join(save_dir, filename_only)

                if os.path.exists(target_path):
                    os.remove(target_path)
                    print(f"🗑 {target_path} 削除")
                else:
                    print(f"⚠ {target_path} は存在しません")

# ボタンに処理を登録
clear_output()
btn_download.on_click(on_download)
btn_delete.on_click(on_delete)

# 表示
display(mlst_dd, widgets.HBox([btn_download, btn_delete]), out)
!rm -rf ~/.cache/huggingface/